In [8]:
import os
import sys
import glob
import scipy
import sklearn
import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
from collections import defaultdict
from matplotlib import pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

%matplotlib inline
sns.set_style('whitegrid')
pd.set_option('display.max_rows', 100)
matplotlib.rcParams['ps.fonttype'] = 42
matplotlib.rcParams['pdf.fonttype'] = 42
pd.set_option('display.max_columns', 100)

sns.set_palette("Set2")
pd.set_option('display.max_colwidth', None)


In [9]:
data_loc = '/home/mattolm/user_data/IndianMicrobiome/tables/'

## Load and incorporate new metadata

In [59]:
data_loc = '/home/mattolm/user_data/IndianMicrobiome/tables/'
Pdb = pd.read_csv(data_loc + 'IndianMicrobiome_basicinfo_v2.csv')
PPdb = Pdb[Pdb['sample_type'] == 'fecal']
PPdb.head()


sample  \
0    AK_SR_1   
1    AK_SR_2   
2  AK_SR_2.2   
3    AK_SR_3   
4    AK_SR_4   

                                                                                                                              read1  \
0      s3://czbiohub-microbiome/Sonnenburg_Lab/IndianMicrobiome/AK_SR_1/03_NODUP/NODUP_HMN_UNMAPPED_TRIM_MARKED_AK_SR_1_R1.fastq.gz   
1      s3://czbiohub-microbiome/Sonnenburg_Lab/IndianMicrobiome/AK_SR_2/03_NODUP/NODUP_HMN_UNMAPPED_TRIM_MARKED_AK_SR_2_R1.fastq.gz   
2  s3://czbiohub-microbiome/Sonnenburg_Lab/IndianMicrobiome/AK_SR_2.2/03_NODUP/NODUP_HMN_UNMAPPED_TRIM_MARKED_AK_SR_2.2_R1.fastq.gz   
3      s3://czbiohub-microbiome/Sonnenburg_Lab/IndianMicrobiome/AK_SR_3/03_NODUP/NODUP_HMN_UNMAPPED_TRIM_MARKED_AK_SR_3_R1.fastq.gz   
4      s3://czbiohub-microbiome/Sonnenburg_Lab/IndianMicrobiome/AK_SR_4/03_NODUP/NODUP_HMN_UNMAPPED_TRIM_MARKED_AK_SR_4_R1.fastq.gz   

                                                                                                                              read2  \
0      s3://czbiohub-microbiome/Sonnenburg_Lab/IndianMicrobiome/AK_SR_1/03_NODUP/NODUP_HMN_UNMAPPED_TRIM_MARKED_AK_SR_1_R2.fastq.gz   
1      s3://czbiohub-microbiome/Sonnenburg_Lab/IndianMicrobiome/AK_SR_2/03_NODUP/NODUP_HMN_UNMAPPED_TRIM_MARKED_AK_SR_2_R2.fastq.gz   
2  s3://czbiohub-microbiome/Sonnenburg_Lab/IndianMicrobiome/AK_SR_2.2/03_NODUP/NODUP_HMN_UNMAPPED_TRIM_MARKED_AK_SR_2.2_R2.fastq.gz   
3      s3://czbiohub-microbiome/Sonnenburg_Lab/IndianMicrobiome/AK_SR_3/03_NODUP/NODUP_HMN_UNMAPPED_TRIM_MARKED_AK_SR_3_R2.fastq.gz   
4      s3://czbiohub-microbiome/Sonnenburg_Lab/IndianMicrobiome/AK_SR_4/03_NODUP/NODUP_HMN_UNMAPPED_TRIM_MARKED_AK_SR_4_R2.fastq.gz   

             library_key  tribe  DNA_concentration sample_type fecal_sample  \
0    LibPlate-A2_AK-SR-1  Kabui               51.5       fecal          NaN   
1    LibPlate-A4_AK-SR-2  Kabui               28.7       fecal          NaN   
2  LibPlate-D2_AK-SR-2-2  Kabui               92.8       fecal          NaN   
3    LibPlate-A6_AK-SR-3  Kabui              411.8       fecal          NaN   
4    LibPlate-A8_AK-SR-4  Kabui               83.3       fecal          NaN   

      pairs       bases  raw_pairs    raw_bases          RL  
0  24276525  6938547858   40041788  11692202096  142.906529  
1  25189547  7192762783   30917233   9027832036  142.772770  
2  11357247  3183231459   20045323   5853234316  140.140980  
3   5682989  1595248519   11410676   3331917392  140.352948  
4   5552061  1578724021    6520759   1904061628  142.174593

In [48]:
SAMPLES = set(PPdb['sample'].tolist())

def calc_sample(row):
    ori = row['sample_name']
    
    if ori.startswith('SK') | ori.startswith('SG') | ori.startswith('SR'):
        samp = 'AK_' + ori.split('_')[0].split('-')[0]
        samp = samp[:5] + '_' + samp[5:]
    
    else:
        samp = ori.split('_')[0].replace('-', '_')
        samp = samp[:5] + '_' + samp[5:]
        
    samp = f"{'_'.join(samp.split('_')[:2])}_{int(samp.split('_')[2])}"
    
    if samp in SAMPLES:
        return samp
    else:
        print(ori, samp)
        #asf
        return 'none'

Mdb = pd.read_csv(data_loc + 'metadata_slack_210927.csv', sep='\t')
Mdb = Mdb[Mdb['description'] == 'stool']
Mdb['sample'] = Mdb.apply(calc_sample, axis=1)
Mdb.head()


AK-SK472_S96 AK_SK_472
AK-SK31_S93 AK_SK_31


sample_name individual_id sample_id description Gender   Age  \
10   AK-SW1_S53            W1     st.W1       stool      F  29.0   
11  AK-SW11_S61           W11    st.W11       stool      M  28.0   
12  AK-SW12_S62           W12    st.W12       stool      M  29.0   
13  AK-SW14_S63           W14    st.W14       stool      M  21.0   
14   AK-SW2_S54            W2     st.W2       stool      F  25.0   

    Weight (kgs)  Height (cm)    BMI  Tribe Village  Run Region  \
10          33.0        159.7  12.94  Warli  Ganjad    1   West   
11          56.0        163.7  20.90  Warli  Ganjad    1   West   
12          51.0        161.2  19.63  Warli  Ganjad    1   West   
13          53.0        162.0  20.20  Warli  Ganjad    1   West   
14          49.0        151.3  21.41  Warli  Ganjad    1   West   

   Fermented_foods Diary_products secondary_cereal Daily_cereal  \
10              No            Yes      Wheat+Jowar         Rice   
11              No            Yes      Wheat+Jowar         Rice   
12              No            Yes      Wheat+Jowar         Rice   
13              No            Yes      Wheat+Jowar         Rice   
14              No            Yes      Wheat+Jowar         Rice   

   Alcoholic_beverage  Terrain    sample  
10                Yes  Plateau   AK_SW_1  
11                Yes  Plateau  AK_SW_11  
12                Yes  Plateau  AK_SW_12  
13                Yes  Plateau  AK_SW_14  
14                Yes  Plateau   AK_SW_2

## Merge

In [63]:
MMdb = Pdb[Pdb['sample_type'] == 'fecal'][['sample', 'library_key', 'tribe']].dropna()
MMdb['sample'] = [x.split('.')[0] for x in MMdb['sample']]
MMdb = MMdb.dropna()
print('MISSING:')
print(set(MMdb['sample']) - set(Mdb['sample']))

MMdb = pd.merge(MMdb, Mdb, how='left')
MMdb.head()


MISSING:
{'AK_SK_24', 'AK_SK_34'}


sample            library_key  tribe sample_name individual_id sample_id  \
0  AK_SR_1    LibPlate-A2_AK-SR-1  Kabui         SR1           SR1   st.t.R1   
1  AK_SR_2    LibPlate-A4_AK-SR-2  Kabui       SR2-2         SR2.2   st.R2.2   
2  AK_SR_2  LibPlate-D2_AK-SR-2-2  Kabui       SR2-2         SR2.2   st.R2.2   
3  AK_SR_3    LibPlate-A6_AK-SR-3  Kabui         SR3           SR3     st.R3   
4  AK_SR_4    LibPlate-A8_AK-SR-4  Kabui         SR4           SR4     st.R4   

  description Gender   Age  Weight (kgs)  Height (cm)  BMI  Tribe  Village  \
0       stool      F  31.0          54.0        143.8  NaN  Kabui  Longsai   
1       stool      M  55.0          53.0        162.0  NaN  Kabui  Longsai   
2       stool      M  55.0          53.0        162.0  NaN  Kabui  Longsai   
3       stool      F  35.0          59.0        144.3  NaN  Kabui  Longsai   
4       stool      M  39.0          55.0        157.2  NaN  Kabui  Longsai   

   Run      Region Fermented_foods Diary_products secondary_cereal  \
0  3.0  North-East             Yes             No             Corn   
1  3.0  North-East             Yes             No             Corn   
2  3.0  North-East             Yes             No             Corn   
3  3.0  North-East             Yes             No             Corn   
4  3.0  North-East             Yes             No             Corn   

  Daily_cereal Alcoholic_beverage Terrain  
0         Rice                Yes   Hills  
1         Rice                Yes   Hills  
2         Rice                Yes   Hills  
3         Rice                Yes   Hills  
4         Rice                Yes   Hills

In [64]:
MMdb[MMdb['tribe'] != MMdb['Tribe']]

sample             library_key    tribe  sample_name individual_id  \
40  AK_SG_12   LibPlate-E20_AK-SG-12    Madia  AK-SG12_S78           G12   
54  AK_SK_24    LibPlate-K8_AK-SK-24   Brokpa          NaN           NaN   
58  AK_SK_49  LibPlate-O6_AK-SK-49-2   Brokpa  AK-SK49_S84           K49   
70  AK_SK_49    LibPlate-O4_AK-SK-49  Purigpa   SK49-2_S43         K49-2   
74  AK_SK_34    LibPlate-M2_AK-SK-34     Boto          NaN           NaN   
76  AK_SK_47   LibPlate-M20_AK-SK-47   Balti*  AK-SK47_S95           K47   
77  AK_SK_47  LibPlate-N2_AK-SK-47-2   Balti*  AK-SK47_S95           K47   
78   AK_SK_8     LibPlate-I8_AK-SK-8   Balti*      SK8_S26            K8   
79  AK_SK_40   LibPlate-M14_AK-SK-40   Balti*     SK40_S51           K40   
80  AK_SK_17    LibPlate-K2_AK-SK-17   Balti*     SK17_S32           K17   
81   AK_SK_5     LibPlate-I6_AK-SK-5   Balti*      SK5_S25            K5   

   sample_id description Gender   Age  Weight (kgs)  Height (cm)    BMI  \
40    st.G12       stool      F  43.0          53.0        155.5  21.92   
54       NaN         NaN    NaN   NaN           NaN          NaN    NaN   
58    st.K49       stool      F  20.0          50.0        152.0  21.64   
70  st.K49-2       stool      M  18.0          60.0        151.5  26.14   
74       NaN         NaN    NaN   NaN           NaN          NaN    NaN   
76    st.K47       stool    NaN   NaN           NaN          NaN    NaN   
77    st.K47       stool    NaN   NaN           NaN          NaN    NaN   
78     st.K8       stool    NaN   NaN           NaN          NaN    NaN   
79    st.K40       stool    NaN   NaN           NaN          NaN    NaN   
80    st.K17       stool    NaN   NaN           NaN          NaN    NaN   
81     st.K5       stool    NaN   NaN           NaN          NaN    NaN   

      Tribe         Village  Run   Region Fermented_foods Diary_products  \
40   Gondia       Hemalkasa  1.0  Central             Yes             No   
54      NaN             NaN  NaN      NaN             NaN            NaN   
58  Purigpa          Kargil  1.0    North             Yes            Yes   
70   Brokpa  Kurtiyal, Dras  2.0    North             Yes            Yes   
74      NaN             NaN  NaN      NaN             NaN            NaN   
76     Boto          Mulbek  1.0    North             Yes            Yes   
77     Boto          Mulbek  1.0    North             Yes            Yes   
78    Balti          Hardas  2.0    North             Yes            Yes   
79    Balti          Hardas  2.0    North             Yes            Yes   
80    Balti          Hardas  2.0    North             Yes            Yes   
81    Balti          Hardas  2.0    North             Yes            Yes   

   secondary_cereal Daily_cereal Alcoholic_beverage  Terrain  
40      Jowar+Bajra         Rice                Yes  Plateau  
54              NaN          NaN                NaN      NaN  
58            Wheat   Rice+Wheat                 No    Hills  
70            Wheat   Rice+Wheat                 No    Hills  
74              NaN          NaN                NaN      NaN  
76     Wheat+Barley   Rice+Wheat                 No    Hills  
77     Wheat+Barley   Rice+Wheat                 No    Hills  
78            Wheat   Rice+Wheat                 No    Hills  
79            Wheat   Rice+Wheat                 No    Hills  
80            Wheat   Rice+Wheat                 No    Hills  
81            Wheat   Rice+Wheat                 No    Hills

# Save metadata

In [65]:
MMdb.to_csv(data_loc + 'IndianMicrobiome_metadata_v1.csv', index=False)